In [1]:
import numpy as np
import pandas as pd

In [25]:
def parse_opt_mp_stream(msg):
    df_mp = pd.DataFrame(msg['data'])
    df_mp.columns = ['eventName', 'time', 'contract', 'markPrice']
    df_mp['time'] = pd.to_datetime(df_mp['time'], unit='ms')
    df_mp = df_mp.pivot(index='time', columns='contract', values='markPrice')
    return df_mp

def parse_opt_index_stream(msg):
    df_idx = pd.DataFrame([msg['data']])
    df_idx.columns = ['eventName', 'time', 'underlying', 'price']
    df_idx['time'] = pd.to_datetime(df_idx['time'], unit='ms')
    
    return df_idx.pivot(index='time', columns='underlying', values='price')

In [23]:
dat = [{'e': 'markPrice', 'E': 1693840301040, 's': 'XRP-230908-0.49-C', 'mp': '1.779'}, {'e': 'markPrice', 'E': 1693840301040, 's': 'XRP-230908-0.49-P', 'mp': '0.904'}, {'e': 'markPrice', 'E': 1693840301040, 's': 'XRP-230908-0.5-C', 'mp': '1.139'}, {'e': 'markPrice', 'E': 1693840301040, 's': 'XRP-230908-0.5-P', 'mp': '1.369'}, {'e': 'markPrice', 'E': 1693840301040, 's': 'XRP-230908-0.51-C', 'mp': '0.694'}]

s,XRPUSDT
E,
1693840301038,0.49877119


In [29]:
import os
BASE_DATA_PATH = "/Users/kibeom/Desktop/coding/crypto_trading/data/"
underlying ='xrp'
symbol = underlying + 'usdt'
contract_info = pd.read_csv(os.path.join(BASE_DATA_PATH, 'contracts.csv'))
contract_info = contract_info[contract_info['underlying'] == symbol.upper()] 


In [34]:
opt_info = {'data': {'e': '24hrTicker', 'E': 1693843338045, 'T': 1693843338000, 's': 'XRP-230908-0.46-C', 'o': '4.83', 'h': '4.83', 'l': '3.88', 'c': '4.54', 'V': '2.7', 'A': '11.9', 'P': '-0.06', 'p': '-0.29', 'Q': '1', 'F': '1', 'L': '4', 'n': 4, 'bo': '3.96', 'ao': '4.87', 'bq': '78', 'aq': '128.5', 'b': '0.0000006', 'a': '1.0485363', 'd': '0.9286384', 't': '-0.00058517', 'g': '4.32919881', 'v': '0.00006871', 'vo': '0.62426815', 'mp': '4.42', 'hl': '8.193', 'll': '0.65', 'eep': '0'}}


In [41]:
raw_data = pd.DataFrame([opt_info['data']])
raw_data = raw_data[['E', 's', 'o', 'h', 'l', 'c', 'V', 'd', 'g', 'v', 'vo', 'bo', 'ao']]
raw_data.columns = ['time', 'contract', 'o', 'h', 'l', 'c', 'v', ]

In [42]:
raw_data

,E,s,o,h,l,c,v,d,g,v,vo,bo,ao
0,1693843338045,XRP-230908-0.46-C,4.83,4.83,3.88,4.54,0.00006871,0.9286384,4.32919881,0.00006871,0.62426815,3.96,4.87
